참고 url : https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb

### version
train_data_size = 225000 (추가학습)   
batch_size = 2    
epochs = 10

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sentencepiece==0.1.91

     |████████████████████████████████| 3.1 MB 15.0 MB/s 
     |████████████████████████████████| 596 kB 60.3 MB/s 
     |████████████████████████████████| 3.3 MB 68.3 MB/s 
     |████████████████████████████████| 59 kB 7.8 MB/s 
     |████████████████████████████████| 895 kB 78.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 13.8 MB/s 


런타임 다시 시작 눌러야할 수 있음  
만일 tokenizer Nonetype 에러시 런타임 다시 시작

In [3]:
# model.generate(pieces)
from transformers import T5Config, T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader



model_folder = '/content/drive/MyDrive/cakd3_colab/3차 프로젝트/Etri_et5/Etri_et5_model_save/Etri_et5_model_save_(180000*10)_extra'

model = T5ForConditionalGeneration.from_pretrained(model_folder)
tokenizer = T5Tokenizer.from_pretrained(model_folder)


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [5]:
class CustomDataset:

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [6]:


def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in tqdm(enumerate(loader, 0)):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]
        
        if _%10 == 0:
            pass
            
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [7]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [8]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(45100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(45100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=768, out_features=3072, bias=False)
              (wi_1): Linear(in_features=768, out_features=3072, bias=False)
              (wo)

hyper-parameters

In [9]:
config = T5Config()
config.MAX_LEN = 1024
config.SUMMARY_LEN = 150 
config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 10        # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1 
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)

In [10]:
train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

In [11]:
!nvidia-smi


Tue Nov 30 21:56:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    39W / 300W |   3015MiB / 16160MiB |     15%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
import pandas as pd


train_dataset = pd.read_csv('/content/drive/MyDrive/cakd3_colab/3차 프로젝트/dataset_split/train_135000_180000.csv')[['document','label']] # 180000개 추가학습에서 train_180000_225000.csv를 사용해서 여기서 train_135000_180000.csv 학습함
validation_dataset = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/valid.csv')[['document','label']]
validation_dataset = validation_dataset.reset_index(drop=True).iloc[:1000]

Training model

In [13]:
train_dataset.columns = ['ctext','text']
train_dataset.ctext = 'summarize: ' + train_dataset.ctext

training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

training_loader = DataLoader(training_set, **train_params)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)
for epoch in range(config.TRAIN_EPOCHS):
    print (1)
    train(epoch, tokenizer, model, device, training_loader, optimizer)

1


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  3.291008234024048


501it [02:42,  3.08it/s]

Epoch: 0, Loss:  3.001330852508545


...

21501it [1:56:21,  3.07it/s]

Epoch: 9, Loss:  0.507166862487793


22001it [1:59:03,  3.07it/s]

Epoch: 9, Loss:  0.4593965709209442


22500it [2:01:45,  3.08it/s]


In [14]:
# model save
tokenizer.save_pretrained('/content/drive/MyDrive/cakd3_colab/3차 프로젝트/Etri_et5/Etri_et5_model_save/Etri_et5_model_save_(225000*10)_extra')
model.save_pretrained('/content/drive/MyDrive/cakd3_colab/3차 프로젝트/Etri_et5/Etri_et5_model_save/Etri_et5_model_save_(225000*10)_extra')

Validation

In [15]:


validation_dataset.columns = ['ctext','text']
validation_dataset.ctext = 'summarize: ' + validation_dataset.ctext

val_set = CustomDataset(validation_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

val_loader = DataLoader(val_set, **val_params)

for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})

final_df

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Completed 0
Completed 100
Completed 200
Completed 300
Completed 400


,Generated Text,Actual Text
0,엔터테인먼트 소속사 커넥트엔터테인먼트가 19일 공식 홈페이지를 통해 악플러 고소 상...,가수 강다니엘이 모욕죄 혐의로 악플러에 대한 고소장을 서울강남경찰서에 제출하며 합의...
1,시의회가 지역 청소년들의 지방의회 역할 이해 및 입법체험을 위해 운영하는 ‘2019...,안산시의회는 지난 7월 22일과 29일에 청소년의회 제1·2차 본회의를 열어 실제 ...
2,경찰서는 지난 14일 오전 11시께 남부대학교 국제수영장 수구 연습경기장에서 몸을 ...,2019 광주세계수영선수권대회에서 18일 일본인A씨가 12명의 여자선수들의 신체 일...
3,건설은 지난 11일 ‘익산 오투그란데 글로벌카운티’ 모델하우스를 오픈하고 총 197...,제일건설은 실수요자들에게 선호도 높은 전용 면적과 효율적인 공간 활용을 강점으로 1...
4,"사업사 에이치엘비 종목은 등락률 +14.87%를 기록하고 있고, 한라IMS 종목은 ...",월요일 11시 00분 현재 조선업종 내 상승종목 등락률 1위를 차지한 에이치엘비종목...
...,...,...
995,법이 할 수 없는 강의를 줄이거나 강사들을 비정년 교원으로 전환해 온 대학들이 '고...,오는 1일부터 '강사법' 시행을 앞둔 가운데 대학들은 고등교육법 개정안에 따른 1년...
996,고를 받은 참전유공자 배우자에게 복지수당을 매월 지급하도록 하는 내용을 담은 제주도...,제주특별자치도 보훈청은 지난 12일 이러한 내용을 담은 제주도 참전유공자 지원 조례...
997,"현현 아산시장은 2019년 시해를 맞아 ""올해는 민선 7기의 실질적인 원년이다""며 ...",오세현 아산시장은 아산의 모든 역량과 기회를 지역경제 활성화에 집중하겠다며 지역경제...
998,지검 여성아동범죄조사부는 2010년 10월에 낳은 여자 유일한 예방접종을 하지 않고...,서울남부지검 여성아동범죄조사부는 2010년 10월에 낳은 여자 아이에게 예방접종을 ...


In [16]:
final_df.to_csv('/content/drive/MyDrive/cakd3_colab/3차 프로젝트/final_df/etri_et5_(225000*10)_extra.csv')

In [17]:
final_df['Generated Text'][4]

'사업사 에이치엘비 종목은 등락률 +14.87%를 기록하고 있고, 한라IMS 종목은 +3.15%를 기록해 조선 업종 내에서 각각 1, 2위를 차지하고 있다.'